In [ ]:
#無人機比賽Training Dataset_v3
#資料前處理(轉換成yolo格式)
from os import listdir
import os
import tqdm
import cv2

pathtxt = 'D:/Can_Work/yolov5-master_AI_CUP_Drone/Drone_data/train'

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)


for root, dirs, files in os.walk(pathtxt):
    for file in files:
        if file.endswith(".png"):
            image = cv2.imread(os.path.join(root, file))
            size = [image.shape[1],image.shape[0]]
        #if file.endswith(".txt"):
            #print(os.path.join(root, file))
            new_line = []
            f = open(os.path.join(root, file.split('.png')[0] + '.txt'), 'r+')
            for line in f.readlines():
                array = line.split(',')
                label = array[0]
                xmin = float(array[1])
                xmax = float(array[1])+float(array[3])
                ymin = float(array[2])
                ymax = float(array[2])+float(array[4])
                
                box = [xmin, xmax, ymin, ymax]
                #box = [array[1], array[2], array[3], array[4]]
                x, y, w, h = convert(size, box)
                #print(x, y, w, h)
                #print(len(line))
                #提取yolo格式類別
                #print(str(label) + ' ' + str(x) + ' '+ str(y) + ' '+ str(abs(w)) + ' '+ str(abs(h)))
                new_line.append(str(label) + ' ' + str(x) + ' ' + str(y) + ' ' + str(abs(w)) + ' ' + str(abs(h)) + '\n')
                #print(new_line)
            f.seek(0)
            f.writelines(new_line)
            f.close()

In [ ]:
#無人機比賽datasets\VisDrone
#資料前處理(刪除及轉換label)
#0 : pedestrian(person)
#1 : people(person)
#2 : bicycle X
#3 : car
#4 : van(car) 
#5 : truck(hov)
#6 : tricycle X
#7 : awning-tricycle X
#8 : bus(hov)
#9 : motor(motorcycle)

#0 : car
#1 : hov
#2 : person
#3 : motorcycle

from os import listdir
import os
import tqdm
import cv2
path = 'D:/Can_Work/yolov5-master_AI_CUP_Drone/datasets/VisDrone/'
for pathtxt in 'VisDrone2019-DET-train/annotations', 'VisDrone2019-DET-val/annotations', 'VisDrone2019-DET-test-dev/annotations':
    for root, dirs, files in os.walk(path + pathtxt):
        for file in files:
            if file.endswith(".txt"):
                #print(os.path.join(root, file))
                new_line = []
                f = open(os.path.join(root, file), 'r')
                f_new = open(os.path.join(path + pathtxt + '_new', file), 'w')

                for line in f.readlines():
                    if(line.split(' ')[0] == str(0)):
                        new_line.append(str(2) + ' ' + line.split(' ', 1)[1])
                    elif(line.split(' ')[0] == str(1)):
                        new_line.append(str(2) + ' ' + line.split(' ', 1)[1])
                    elif(line.split(' ')[0] == str(3)):
                        new_line.append(str(0) + ' ' + line.split(' ', 1)[1])
                    elif(line.split(' ')[0] == str(4)):
                        new_line.append(str(0) + ' ' + line.split(' ', 1)[1])
                    elif(line.split(' ')[0] == str(5)):
                        new_line.append(str(1) + ' ' + line.split(' ', 1)[1])
                    elif(line.split(' ')[0] == str(8)):
                        new_line.append(str(1) + ' ' + line.split(' ', 1)[1])
                    elif(line.split(' ')[0] == str(9)):
                        new_line.append(str(3) + ' ' + line.split(' ', 1)[1])
                f_new.seek(0)
                f_new.writelines(new_line)
                f.close()
                f_new.close()

In [ ]:
#!pip install sahi
#!pip install yolov5
#!pip install pandas

In [ ]:
from sahi.model import Yolov5DetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image
from os import listdir
from PIL import Image as PImage
import pandas as pd
from tqdm import tqdm, trange

In [ ]:
yolov5_model_path = 'best.pt'

#載入model
detection_model = Yolov5DetectionModel(
    model_path=yolov5_model_path,
    confidence_threshold=0.3,
    device='cuda:0', # or 'cpu'
)
#download_from_url('https://raw.githubusercontent.com/obss/sahi/main/demo/demo_data/small-vehicles1.jpeg', 'demo_data/small-vehicles1.jpeg')
#download_from_url('https://raw.githubusercontent.com/obss/sahi/main/demo/demo_data/terrain2.png', 'demo_data/terrain2.png')

#要偵測的圖片路徑
path = "Drone_data/Public_Testing_Dataset_v2/"
imagesList = listdir(path)
#print(imagesList)

In [ ]:
count = 0
result_list = []
for img_path in imagesList:
    result = get_sliced_prediction(
        "Drone_data/Public_Testing_Dataset_v2/" + img_path,
        detection_model,
        slice_height = 512,
        slice_width = 512,
        overlap_height_ratio = 0.2,
        overlap_width_ratio = 0.2
    )
    
    result.export_visuals(export_dir="export_dir/Public_Testing_Dataset_v2/" + "prediction_visual_" + str(count) + ".png")
    count = count + 1
    
    object_prediction_list = result.object_prediction_list
    for i in range(len(object_prediction_list)):
        #print('coordinate: ' + str(object_prediction_list[i].bbox))
        #print('label: ' + str(object_prediction_list[i].category))
        result_list.append([img_path.split('.')[0], 
                            str(object_prediction_list[i].category), 
                            #object_prediction_list[i].bbox,
                            str(object_prediction_list[i].bbox).split('(')[1].split(')')[0].replace(" ", "").split(',')[0],
                            str(object_prediction_list[i].bbox).split('(')[1].split(')')[0].replace(" ", "").split(',')[1],
                            str(object_prediction_list[i].bbox).split('(')[1].split(')')[0].replace(" ", "").split(',')[2],
                            str(object_prediction_list[i].bbox).split('(')[1].split(')')[0].replace(" ", "").split(',')[3]])
                            #str(object_prediction_list[i].bbox).split('(')[1].split(')')[0].replace(" ", "")])
    #print(result_list)

In [ ]:
columns = ["img_name", "predict_label",
           "predict_bounding_box_x", "predict_bounding_box_y", "predict_bounding_box_w", "predict_bounding_box_h"]
dt = pd.DataFrame(result_list, columns=columns)
dt.to_csv("result_csv.csv", index=0)

#輸出csv後記得開csv檔刪除第一列標題

In [ ]:
    object_prediction_list = result.object_prediction_list
    for i in range(len(object_prediction_list)):
        #print('coordinate: ' + str(object_prediction_list[i].bbox))
        #print('label: ' + str(object_prediction_list[i].category))
        tmp_bbox = [str(object_prediction_list[i].bbox).split('(')[1].split(')')[0].replace(" ", "").split(',')[0]
             ,str(object_prediction_list[i].bbox).split('(')[1].split(')')[0].replace(" ", "").split(',')[1]
             ,str(object_prediction_list[i].bbox).split('(')[1].split(')')[0].replace(" ", "").split(',')[2]
             ,str(object_prediction_list[i].bbox).split('(')[1].split(')')[0].replace(" ", "").split(',')[3]]
        tmp_bbox_2 = ','.join(str(j) for j in tmp_bbox)
        #print(tmp_bbox_2)
        result_list.append([img_path.split('.')[0], 
                            str(object_prediction_list[i].category), 
                            tmp_bbox_2])